In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [2]:
out='../output/'
inb='../input/flows/'
pref='dam'
inp = inb + pref + '/'

In [3]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2016-01-01,160,ZAMZ,0,ZAMC
1,2016-01-01,170,ZIM,0,ZAMZ
2,2016-01-01,160,ZAMZ,1,ZAMC
3,2016-01-01,170,ZIM,1,ZAMZ
4,2016-01-01,160,ZAMZ,2,ZAMC
...,...,...,...,...,...
5647,2022-10-31,86,RSAS,23,NAM
5648,2022-10-31,82,ZAMZ,23,ZAMC
5649,2022-10-31,35,ZAMZ,23,ZIM
5650,2022-10-31,35,ZIM,23,ZIMA


In [4]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

C:\Users\kabom\AppData\Local\Temp\ipykernel_17548\1849715270.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dam=dam.drop(['date'],1)


price  volume  year  month  day  day_of_week      value  \
date       hour                                                             
2016-01-01 0      40.10   477.5  2016      1    1            4  19147.750   
           1      40.08   486.6  2016      1    1            4  19502.928   
           2      40.06   495.6  2016      1    1            4  19853.736   
           3      40.08   496.6  2016      1    1            4  19903.728   
           4      40.07   502.9  2016      1    1            4  20151.203   
...                 ...     ...   ...    ...  ...          ...        ...   
2022-10-31 19    310.08    92.0  2022     10   31            0  28527.360   
           20    310.04   156.0  2022     10   31            0  48366.240   
           21    223.09   162.0  2022     10   31            0  36140.580   
           22    126.34   123.0  2022     10   31            0  15539.820   
           23     68.56   168.0  2022     10   31            0  11518.080   

                      cat  
date       hour            
2016-01-01 0          off  
           1          off  
           2          off  
           3          off  
           4          off  
...                   ...  
2022-10-31 19     evening  
           20    standard  
           21    standard  
           22         off  
           23         off  

[59104 rows x 8 columns]

In [5]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

C:\Users\kabom\AppData\Local\Temp\ipykernel_17548\2288514685.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  flow=flow.dropna(0)


,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2016-01-01,0,160,ZAMZ,ZAMC,2016,1,1,4,40.10,6416.00
1,2016-01-01,0,170,ZIM,ZAMZ,2016,1,1,4,40.10,6817.00
2,2016-01-01,1,160,ZAMZ,ZAMC,2016,1,1,4,40.08,6412.80
3,2016-01-01,1,170,ZIM,ZAMZ,2016,1,1,4,40.08,6813.60
4,2016-01-01,2,160,ZAMZ,ZAMC,2016,1,1,4,40.06,6409.60
...,...,...,...,...,...,...,...,...,...,...,...
435813,2022-10-31,23,86,RSAS,NAM,2022,10,31,0,68.56,5896.16
435814,2022-10-31,23,82,ZAMZ,ZAMC,2022,10,31,0,68.56,5621.92
435815,2022-10-31,23,35,ZAMZ,ZIM,2022,10,31,0,68.56,2399.60
435816,2022-10-31,23,35,ZIM,ZIMA,2022,10,31,0,68.56,2399.60


In [6]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2016-01-01,0,160,6416.00,40.10,ZAMC,2016,1,1,4
1,2016-01-01,0,170,6817.00,40.10,ZAMZ,2016,1,1,4
2,2016-01-01,1,160,6412.80,40.08,ZAMC,2016,1,1,4
3,2016-01-01,1,170,6813.60,40.08,ZAMZ,2016,1,1,4
4,2016-01-01,2,160,6409.60,40.06,ZAMC,2016,1,1,4
...,...,...,...,...,...,...,...,...,...,...
435813,2022-10-31,23,-86,-5896.16,68.56,RSAS,2022,10,31,0
435814,2022-10-31,23,-82,-5621.92,68.56,ZAMZ,2022,10,31,0
435815,2022-10-31,23,-35,-2399.60,68.56,ZAMZ,2022,10,31,0
435816,2022-10-31,23,-35,-2399.60,68.56,ZIM,2022,10,31,0


In [ ]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [ ]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

In [ ]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

In [ ]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')